Libraries

In [1]:
import pygame
import sys
import random
import time
import serial
import numpy as np
from numpy import sin, cos
from vpython import *
import random

pygame 2.5.2 (SDL 2.28.3, Python 3.11.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


<IPython.core.display.Javascript object>

Initialize Pygame

In [2]:
pygame.init()

(5, 0)

Constants

In [3]:
WIDTH, HEIGHT = 600, 600
CELL_SIZE = 30
ROWS, COLS = HEIGHT // CELL_SIZE, WIDTH // CELL_SIZE
FPS = 300  # Adjust the FPS for slower movement

Colors

In [4]:
WHITE = (255, 255, 255)
GRAY = (181, 176, 173)
DARK_BLUE = (3, 102, 150)
MEDIUM_BLUE = (7, 87, 91)
LIGHT_BLUE = (173, 216, 230)
RED = (255, 0, 0)

Initialize clock

In [5]:
clock = pygame.time.Clock()

Initialize serial communication with Arduino

In [6]:
arduinoData = serial.Serial('com9', 115200)
time.sleep(1)

Maze generation - DO NOT CHANGE THIS FUNCTION!

In [7]:
def generate_maze():
    maze = [[0] * COLS for _ in range(ROWS)]
    stack = []
    start_cell = (1, 1)
    end_cell = (ROWS - 3, COLS - 3)
    stack.append(start_cell)
    maze[start_cell[0]][start_cell[1]] = 1

    while stack:
        current_cell = stack[-1]
        neighbors = [
            (current_cell[0] - 2, current_cell[1]),
            (current_cell[0] + 2, current_cell[1]),
            (current_cell[0], current_cell[1] - 2),
            (current_cell[0], current_cell[1] + 2),
        ]
        unvisited_neighbors = [neighbor for neighbor in neighbors if 0 < neighbor[0] < ROWS - 1 and 0 < neighbor[1] < COLS - 1 and maze[neighbor[0]][neighbor[1]] == 0]

        if unvisited_neighbors:
            chosen_neighbor = random.choice(unvisited_neighbors)
            maze[chosen_neighbor[0]][chosen_neighbor[1]] = 1
            maze[(chosen_neighbor[0] + current_cell[0]) // 2][(chosen_neighbor[1] + current_cell[1]) // 2] = 1
            stack.append(chosen_neighbor)
        else:
            stack.pop()

    return maze, start_cell, end_cell

maze, start_point, end_point = generate_maze()

# Player position
player_row, player_col = start_point

Game

In [8]:
# Initialize the screen
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Maze Game")

# Game loop
while True:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()
    try:
    # Read data from MPU-6050 sensor through serial communication
        toRad = np.pi / 180.0
        toDeg = 1 / toRad
        while arduinoData.inWaiting() == 0:
            pass
        data_packet = arduinoData.readline()
        data_packet = str(data_packet, 'utf-8')
        split_packet = data_packet.split(",")
        roll = float(split_packet[0]) * toRad * toDeg
        pitch = float(split_packet[1]) * toRad * toDeg
        yaw = float(split_packet[2]) * toRad * toDeg
        
    # Adjust player position based on MPU-6050 data
    
        if pitch>20 and player_col > 0 and maze[player_row][player_col - 1] == 1:
            player_col -= 1
        if pitch<-20 and player_col < COLS - 1 and maze[player_row][player_col + 1] == 1:
            player_col += 1
        if roll<-20 and player_row > 0 and maze[player_row - 1][player_col] == 1:
            player_row -= 1
        if roll>20 and player_row < ROWS - 1 and maze[player_row + 1][player_col] == 1:
            player_row += 1
        
    
        
    except Exception as e:
        print("Error:", e)

    # Draw maze
    screen.fill(WHITE)
    for row in range(ROWS):
        for col in range(COLS):
            if maze[row][col] == 1:
                pygame.draw.rect(screen, MEDIUM_BLUE, (col * CELL_SIZE, row * CELL_SIZE, CELL_SIZE, CELL_SIZE))
    for row in range(ROWS):
        for col in range(COLS):
            if (row, col) == start_point:
                pygame.draw.rect(screen, LIGHT_BLUE, (col * CELL_SIZE, row * CELL_SIZE, CELL_SIZE, CELL_SIZE))
            elif (row, col) == end_point:
                pygame.draw.rect(screen, LIGHT_BLUE, (col * CELL_SIZE, row * CELL_SIZE, CELL_SIZE, CELL_SIZE))

    # Draw player
    pygame.draw.circle(screen, GRAY, (player_col * CELL_SIZE + CELL_SIZE // 2, player_row * CELL_SIZE + CELL_SIZE // 2), CELL_SIZE // 3)
    
    # Check if the player reaches the end point
    if (player_row, player_col) == end_point:
        pygame.display.flip()
        print("Congratulations! You reached the end of the maze.")
        time.sleep(5)
        pygame.quit()
        sys.exit()
    pygame.display.flip()
    clock.tick(FPS)

Error: could not convert string to float: 'Initializing I2C devices...\r\n'
Error: could not convert string to float: 'Testing device connections...\r\n'
Error: could not convert string to float: 'MPU6050 connection successful\r\n'
Error: could not convert string to float: 'Initializing DMP...\r\n'
Error: could not convert string to float: '>*......>......-4372.00000'
Error: could not convert string to float: '\n'
Error: could not convert string to float: 'Enabling DMP...\r\n'
Congratulations! You reached the end of the maze.


SystemExit: 

C:\ProgramData\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3513: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
